# 1. Import Library

In [1]:
import os
import csv
import pandas as pd
import numpy as np
import random
import time
import datetime
import json
import os
import math
import tensorflow as tf
import torch
from torch import nn
from keras import models
from keras import layers
from kobert import get_pytorch_kobert_model

from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers.activations import gelu, gelu_new

from transformers.modeling_utils import PreTrainedModel, prune_linear_layer, find_pruneable_heads_and_indices
from transformers.file_utils import add_start_docstrings_to_model_forward, add_code_sample_docstrings
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
from yellowbrick.regressor import RegressionScoreVisualizer
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ClassificationReport
from sklearn import datasets, metrics
from torch.nn import CrossEntropyLoss


# new_Added
from transformers.models.bert.modeling_bert import BertSelfOutput, BertIntermediate, BertOutput, BertPooler, BertSelfAttention, BertAttention, BertPreTrainedModel, BertEncoder, BertLayer
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거


2022-06-18 21:36:08.537463: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def swish(x):
    return x * tf.sigmoid(x)

def mish(x):
    return x * torch.tanh(nn.functional.softplus(x))

In [3]:
ACT2FN = {"gelu": gelu, "relu": torch.nn.functional.relu, "swish": swish, "gelu_new": gelu_new, "mish": mish}

BertLayerNorm = torch.nn.LayerNorm
BERT_INPUTS_DOCSTRING = r"""
    Args:
        input_ids (:obj:`Numpy array` or :obj:`tf.Tensor` of shape :obj:`{0}`):
            Indices of input sequence tokens in the vocabulary.

            Indices can be obtained using :class:`transformers.BertTokenizer`.
            See :func:`transformers.PreTrainedTokenizer.encode` and
            :func:`transformers.PreTrainedTokenizer.__call__` for details.

            `What are input IDs? <../glossary.html#input-ids>`__
        attention_mask (:obj:`Numpy array` or :obj:`tf.Tensor` of shape :obj:`{0}`, `optional`, defaults to :obj:`None`):
            Mask to avoid performing attention on padding token indices.
            Mask values selected in ``[0, 1]``:
            ``1`` for tokens that are NOT MASKED, ``0`` for MASKED tokens.

            `What are attention masks? <../glossary.html#attention-mask>`__
        token_type_ids (:obj:`Numpy array` or :obj:`tf.Tensor` of shape :obj:`{0}`, `optional`, defaults to :obj:`None`):
            Segment token indices to indicate first and second portions of the inputs.
            Indices are selected in ``[0, 1]``: ``0`` corresponds to a `sentence A` token, ``1``
            corresponds to a `sentence B` token

            `What are token type IDs? <../glossary.html#token-type-ids>`__
        position_ids (:obj:`Numpy array` or :obj:`tf.Tensor` of shape :obj:`{0}`, `optional`, defaults to :obj:`None`):
            Indices of positions of each input sequence tokens in the position embeddings.
            Selected in the range ``[0, config.max_position_embeddings - 1]``.

            `What are position IDs? <../glossary.html#position-ids>`__
        head_mask (:obj:`Numpy array` or :obj:`tf.Tensor` of shape :obj:`(num_heads,)` or :obj:`(num_layers, num_heads)`, `optional`, defaults to :obj:`None`):
            Mask to nullify selected heads of the self-attention modules.
            Mask values selected in ``[0, 1]``:
            :obj:`1` indicates the head is **not masked**, :obj:`0` indicates the head is **masked**.
        inputs_embeds (:obj:`Numpy array` or :obj:`tf.Tensor` of shape :obj:`(batch_size, sequence_length, embedding_dim)`, `optional`, defaults to :obj:`None`):
            Optionally, instead of passing :obj:`input_ids` you can choose to directly pass an embedded representation.
            This is useful if you want more control over how to convert `input_ids` indices into associated vectors
            than the model's internal embedding lookup matrix.
        training (:obj:`boolean`, `optional`, defaults to :obj:`False`):
            Whether to activate dropout modules (if set to :obj:`True`) during training or to de-activate them
            (if set to :obj:`False`) for evaluation.
        output_attentions (:obj:`bool`, `optional`, defaults to :obj:`None`):
            If set to ``True``, the attentions tensors of all attention layers are returned. See ``attentions`` under returned tensors for more detail.
"""

_TOKENIZER_FOR_DOC = "BertTokenizer"

# 2. Device Check

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [5]:
if torch.cuda.is_available():
    device = "cuda"
    print("There are some GPU which can be used")
    
else:
    device = "cpu"
    print("There is no GPU which can be used")
        
print(torch.rand(10).to(device))


There are some GPU which can be used
tensor([0.2246, 0.0868, 0.8848, 0.8563, 0.0572, 0.7038, 0.1495, 0.5259, 0.2179,
        0.4351], device='cuda:0')


# 3. Model & Function Define

In [6]:
class BertEmbeddings_revised(nn.Module):
    """Construct the embeddings from word, position and token_type embeddings.
    """

    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)
        self.turn_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)

        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids=None, token_type_ids=None, position_ids=None, turn_ids=None, inputs_embeds=None):
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]

        seq_length = input_shape[1]
        device = input_ids.device if input_ids is not None else inputs_embeds.device
        if position_ids is None:
            position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
            position_ids = position_ids.unsqueeze(0).expand(input_shape)
            
        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        if inputs_embeds is None:
            inputs_embeds = self.word_embeddings(input_ids)
            
        if turn_ids is None:
            turn_ids = torch.zeros(input_shape, dtype=torch.long, device=device)
            for i in range(len(input_ids)):
                temp = 2
                for j in range(len(input_ids[i])):
                    if (input_ids[i][j] == 101 or input_ids[i][j] == 102 or input_ids[i][j] == 0):
                        if temp == 1:
                            temp = 2
                        elif temp == 2:
                            temp = 1
                        turn_ids[i][j] = 0
                        continue
                    turn_ids[i][j] = temp
        #turn_ids = turn_ids.clone().detach().long()
        #turn_ids = torch.tensor(turn_ids).to(device).long()
        #turn_ids=turn_ids.type(torch.LongTensor)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        turn_embeddings = self.turn_embeddings(turn_ids)

        embeddings = inputs_embeds + position_embeddings + token_type_embeddings + turn_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [7]:
class BertModel_revised(BertPreTrainedModel):
    """

    The model can behave as an encoder (with only self-attention) as well
    as a decoder, in which case a layer of cross-attention is added between
    the self-attention layers, following the architecture described in `Attention is all you need`_ by Ashish Vaswani,
    Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser and Illia Polosukhin.

    To behave as an decoder the model needs to be initialized with the
    :obj:`is_decoder` argument of the configuration set to :obj:`True`; an
    :obj:`encoder_hidden_states` is expected as an input to the forward pass.

    .. _`Attention is all you need`:
        https://arxiv.org/abs/1706.03762

    """

    def __init__(self, config):
        super().__init__(config)
        self.config = config

        self.embeddings = BertEmbeddings_revised(config)
        self.encoder = BertEncoder(config)
        self.pooler = BertPooler(config)

        self.init_weights()
        
    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value
            
    def _prune_heads(self, heads_to_prune):
        """ Prunes heads of the model.
            heads_to_prune: dict of {layer_num: list of heads to prune in this layer}
            See base class PreTrainedModel
        """
        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)
    
    @add_start_docstrings_to_model_forward(BERT_INPUTS_DOCSTRING.format("(batch_size, sequence_length)"))
    @add_code_sample_docstrings(processor_class=_TOKENIZER_FOR_DOC, checkpoint="bert-base-uncased")
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        turn_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        output_attentions=None,
        output_hidden_states=None,
    ):
        r"""
    Return:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.BertConfig`) and inputs:
        last_hidden_state (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length, hidden_size)`):
            Sequence of hidden-states at the output of the last layer of the model.
        pooler_output (:obj:`torch.FloatTensor`: of shape :obj:`(batch_size, hidden_size)`):
            Last layer hidden-state of the first token of the sequence (classification token)
            further processed by a Linear layer and a Tanh activation function. The Linear
            layer weights are trained from the next sentence prediction (classification)
            objective during pre-training.

            This output is usually *not* a good summary
            of the semantic content of the input, you're often better with averaging or pooling
            the sequence of hidden-states for the whole input sequence.
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``output_hidden_states=True`` is passed or when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``output_attentions=True`` is passed or when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        device = input_ids.device if input_ids is not None else inputs_embeds.device

        if attention_mask is None:
            attention_mask = torch.ones(input_shape, device=device)
        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)
        if turn_ids is None:
            turn_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape, device)

        # If a 2D ou 3D attention mask is provided for the cross-attention
        # we need to make broadcastabe to [batch_size, num_heads, seq_length, seq_length]
        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            input_ids=input_ids, position_ids=position_ids, token_type_ids=token_type_ids, turn_ids=turn_ids, inputs_embeds=inputs_embeds
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output)

        outputs = (sequence_output, pooled_output,) + encoder_outputs[
            1:
        ]  # add hidden_states and attentions if they are here
        return outputs  # sequence_output, pooled_output, (hidden_states), (attentions)


In [8]:
class BertForSequenceClassification_revised(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel_revised(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    @add_start_docstrings_to_model_forward(BERT_INPUTS_DOCSTRING.format("(batch_size, sequence_length)"))
    @add_code_sample_docstrings(processor_class=_TOKENIZER_FOR_DOC, checkpoint="bert-base-uncased")
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        turn_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            turn_ids=turn_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # loss, logits, (hidden_states), (attentions)


In [9]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = preds.flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


def evaluation_metrics(preds, labels):
    pred_flat = preds.flatten()
    labels_flat = labels.flatten()
    
    accuracy=accuracy_score(pred_flat, labels_flat)
    precision=precision_score(pred_flat, labels_flat, average='macro')
    recall=recall_score(pred_flat, labels_flat, average='macro')
    f1=f1_score(pred_flat, labels_flat, average='macro')
    RMSE = mean_squared_error(labels_flat, pred_flat)**0.5
    
    return accuracy, precision, recall, f1, RMSE

def softmax(num):
    exp=np.exp(num)
    sum_exp=np.sum(exp)
    y=exp/sum_exp
    
    return y

# 4. Chatbot Data

## 4.1 Data Preprocessing

In [10]:
chatbot_data=pd.read_csv('ChatbotData.csv', encoding='cp949')

In [11]:
chatbot_data.head(5)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [12]:
input_list=[]

for idx,row in chatbot_data.iterrows():
    Q=row['Q']
    A=row['A']
    input="[CLS] "+Q+" [SEP] "+A+" [SEP]"
    input_list.append(input)

In [13]:
label=chatbot_data['label']

In [14]:
label_list=label.values.tolist()

In [15]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
tokenized_input = [tokenizer.tokenize(input) for input in input_list]

In [16]:
print(input_list[0])
print(tokenized_input[0])

[CLS] 12시 땡! [SEP] 하루가 또 가네요. [SEP]
['[CLS]', '12', '##시', '[UNK]', '!', '[SEP]', '하', '##루', '##가', '또', '가', '##네', '##요', '.', '[SEP]']


In [17]:
MAX_LEN = 128 

input_id = [tokenizer.convert_tokens_to_ids(token) for token in tokenized_input]
input_id = pad_sequences(input_id, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [18]:
attention_masks = []

for num in input_id:
    attention_mask = [float(i>0) for i in num]
    attention_masks.append(attention_mask)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(input_id, label_list, random_state=2, test_size=0.4, shuffle=True)
mask_train, mask_test, temp_train_id, temp_test_id = train_test_split(attention_masks, input_id, random_state=2, test_size=0.4, shuffle=True)     

In [20]:
x_val, x_eval, y_val, y_eval = train_test_split(x_test, y_test, test_size=0.2, random_state=2, shuffle=True)
mask_val, mask_eval, _, _ = train_test_split(mask_test, temp_test_id, random_state=2, test_size=0.2, shuffle=True)   

In [21]:
train_inputs = torch.tensor(x_train)
train_labels = torch.tensor(y_train)
train_masks = torch.tensor(mask_train)

val_inputs = torch.tensor(x_val)
val_labels = torch.tensor(y_val)
val_masks = torch.tensor(mask_val)  

eval_inputs1 = torch.tensor(x_eval)
eval_labels1 = torch.tensor(y_eval)
eval_masks1 = torch.tensor(mask_eval)  

In [22]:
BATCH_SIZE = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE)

eval_data1 = TensorDataset(eval_inputs1, eval_masks1, eval_labels1)
eval_sampler1 = SequentialSampler(eval_data1)
eval_dataloader1 = DataLoader(eval_data1, sampler=eval_sampler1, batch_size=BATCH_SIZE)

## 4.2 Training

In [23]:
#SemanticScorePrediction = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
SemanticScorePrediction = BertForSequenceClassification_revised.from_pretrained("bert-base-multilingual-cased", num_labels=3)
SemanticScorePrediction.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification_revised: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification_revised from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification_revised from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification_revised were n

BertForSequenceClassification_revised(
  (bert): BertModel_revised(
    (embeddings): BertEmbeddings_revised(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (turn_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias

In [24]:
optimizer = AdamW(SemanticScorePrediction.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 30

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [25]:
#랜덤시드 고정
seed_val = 14
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
SemanticScorePrediction.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    SemanticScorePrediction.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        for i in range(len(b_labels)):
            b_labels[i] += 1

        # Forward 수행
        outputs = SemanticScorePrediction(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask,
                        turn_ids = None,
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(SemanticScorePrediction.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        SemanticScorePrediction.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))


======== Epoch 1 / 30 ========
Training...

  Average training loss: 0.68
  Training epcoh took: 0:00:34

======== Epoch 2 / 30 ========
Training...

  Average training loss: 0.44
  Training epcoh took: 0:00:32

======== Epoch 3 / 30 ========
Training...

  Average training loss: 0.34
  Training epcoh took: 0:00:33

======== Epoch 4 / 30 ========
Training...

  Average training loss: 0.26
  Training epcoh took: 0:00:33

======== Epoch 5 / 30 ========
Training...

  Average training loss: 0.20
  Training epcoh took: 0:00:33

======== Epoch 6 / 30 ========
Training...

  Average training loss: 0.16
  Training epcoh took: 0:00:33

======== Epoch 7 / 30 ========
Training...

  Average training loss: 0.14
  Training epcoh took: 0:00:33

======== Epoch 8 / 30 ========
Training...

  Average training loss: 0.11
  Training epcoh took: 0:00:33

======== Epoch 9 / 30 ========
Training...

  Average training loss: 0.09
  Training epcoh took: 0:00:33

======== Epoch 10 / 30 ========
Training...



## 4.3 Validation

In [49]:
def relation_score_prediction(logtis):
    logits_numpy=logits.detach().cpu().numpy()
    indicator_list=[]
    score_list=[]
    
    Input_layer = layers.InputLayer(input_shape=(1,))
    hidden_layer1 = layers.Dense(8, activation='swish')
    hidden_layer2 = layers.Dense(4, activation='swish')
    output_layer = layers.Dense(1, activation='tanh')
    
    model = tf.keras.Sequential([
        Input_layer,
        hidden_layer1,
        hidden_layer2,
        output_layer,
    ])

    model.compile(optimizer='Adam', loss='mse')
    
    for i in range(len(logits)):
        j = np.argmax(logits_numpy[i])
        k = np.argmin(logits_numpy[i])
        ground_truth = np.zeros(3)
        ground_truth[j] = 5
        ground_truth[k] = -5
        model.fit(logits_numpy[i], ground_truth)
        
        
    for i in range(len(logits)):
        semantic_indicator=model.predict(logits_numpy[i])
        indicator_list.append(semantic_indicator)
        
    for i in range(len(indicator_list)):
        indicator_list[i][0]=(-1)*indicator_list[i][0]
        sum=0
        for j in range(len(indicator_list[i])):
            sum=sum+indicator_list[i][j]
        sum=sum/3
        score_list.append(sum)
            
    return score_list

In [27]:
#Validation
#시작 시간 설정
t0 = time.time()

logits_results = np.array([])
label_ids_results = np.array([])

#데이터로더에서 배치만큼 반복하여 가져옴
for batch in val_dataloader:
    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)

    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch

    for i in range(len(b_labels)):
        b_labels[i] += 1

    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = SemanticScorePrediction(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask,
                        turn_ids = None,
                        labels=b_labels)
    #로짓 구함
    logits = outputs[1]
    
    
    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy() 
    
    logits = np.argmax(logits, axis=1)

    logits_results = np.append(logits_results, logits)
    label_ids_results = np.append(label_ids_results, label_ids)
    

# 출력 로스과 라벨을 비교하여 정확도 계산
tmp_eval_accuracy = flat_accuracy(logits_results, label_ids_results)
#eval_accuracy += tmp_eval_accuracy
#nb_eval_steps += 1
acc, prec, recall, f1, RMSE = evaluation_metrics(logits_results, label_ids_results)

print('  [Accuracy]: {:.2f}%'.format(acc * 100))
print('  [Precision]: {:.2f}%'.format(prec * 100))
print('  [Recall]: {:.2f}%'.format(recall * 100))
print('  [F1_Score]: {:.2f}%'.format(f1 * 100))
print('  [RMSE]: {:.2f}'.format(RMSE))
print("  Validation took: {:}".format(format_time(time.time() - t0)))

  [Accuracy]: 86.42%
  [Precision]: 85.87%
  [Recall]: 86.55%
  [F1_Score]: 86.17%
  [RMSE]: 0.46
  Validation took: 0:00:05


## 4.4 Evaluation

In [28]:
#Evaluation
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
SemanticScorePrediction.eval()

score_list_final=[]
# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
logits_results = np.array([])
label_ids_results = np.array([])

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(eval_dataloader1):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(eval_dataloader1), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    for i in range(len(b_labels)):
            b_labels[i] += 1
        
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = SemanticScorePrediction(b_input_ids, 
                        token_type_ids=None, 
                        turn_ids=None,
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]
    #score_list=relation_score_prediction(logits)
    #score_list_final.apppend(score_list)
    
    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    logits = np.argmax(logits, axis=1)

    logits_results = np.append(logits_results, logits)
    label_ids_results = np.append(label_ids_results, label_ids)
    

# 출력 로스과 라벨을 비교하여 정확도 계산
tmp_eval_accuracy = flat_accuracy(logits_results, label_ids_results)
eval_accuracy += tmp_eval_accuracy
nb_eval_steps += 1
acc, prec, recall, f1, RMSE = evaluation_metrics(logits_results, label_ids_results)

print('  [Accuracy]: {:.2f}%'.format(acc * 100))
print('  [Precision]: {:.2f}%'.format(prec * 100))
print('  [Recall]: {:.2f}%'.format(recall * 100))
print('  [F1_Score]: {:.2f}%'.format(f1 * 100))
print('  [RMSE]: {:.2f}'.format(RMSE))
print("  Evaluation  took: {:}".format(format_time(time.time() - t0)))


  [Accuracy]: 86.89%
  [Precision]: 86.14%
  [Recall]: 86.71%
  [F1_Score]: 86.40%
  [RMSE]: 0.47
  Evaluation  took: 0:00:01


## 5. Everyone's corpus: Messenger 

## 5.1 Data Preprocessing

In [29]:
path = 'NIKL_OM_2021_v1.0/NIKL_OM_2021_v1.0/국립국어원 온라인 대화 말뭉치 2021(버전 1.0)/'
file_list = os.listdir(path)
file_list_json = [file for file in file_list if file.endswith('.json')] 

In [30]:
data_dict={}
data_arr=[]

num=0
for file in file_list_json:
    input_data="[CLS]"
    temp='1'
    with open (path+file, "r", encoding="UTF8") as f:
        contents=f.read()
        json_data=json.loads(contents)
        relation=json_data["document"][0]["metadata"]["setting"]["relation"]
        utter=json_data["document"][0]["utterance"]
        speaker1_gender=json_data["document"][0]["metadata"]["speaker"][0]["sex"]
        speaker2_gender=json_data["document"][0]["metadata"]["speaker"][1]["sex"]
        if(speaker1_gender==speaker2_gender):
            label=0
        else:
            label="none"
        
        for utter_num in range(len(utter)):
            if temp==utter[utter_num]['speaker_id']:
                input_data=input_data+" "+utter[utter_num]["form"]
                temp=utter[utter_num]['speaker_id']
            else:
                input_data=input_data+" [SEP] "+utter[utter_num]["form"]
                temp=utter[utter_num]['speaker_id']
                
        input_data=input_data+" [SEP] "
        
        num=num+1
        if(num%10000==0):
            print(num)
            
        data_dict={
            'data':input_data,
            'label':label,
            'relation':relation,
            'speaker1_gender':speaker1_gender,
            'speaker2_gender':speaker2_gender
        }
        data_arr.append(data_dict)

10000
20000
30000
40000


In [31]:
with open('malmungchi_chatbot.csv','w',encoding='utf-8-sig') as csv_file:
    fieldnames = ['data', 'label','relation','speaker1_gender','speaker2_gender']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for data in data_arr:
        writer.writerow(data)

In [32]:
malmungchi_chatbot_data=pd.read_csv('malmungchi_chatbot.csv')

In [33]:
malmungchi_4000_samples=pd.read_csv('malmugchi_chabot_4000_samples.csv', encoding='cp949')

In [34]:
malmungchi_4000_samples.head()

,data,label,relation,speaker1_gender,speaker2_gender
0,[CLS] 전 이제 집 도착해서 저녁먹었는데 저녁 드셨나요?ㅎㅎ [SEP] 네 게 ...,0,낯선 사람,여성,여성
1,[CLS] 안녕 [SEP] 나 옷 삼 [SEP] 뭔 옷 [SEP] 에이블리에서 후드...,0,가족>형제자매,여성,여성
2,[CLS] [SEP] 저는 홍차! [SEP] 아아vs뜨아? [SEP] 아아요! 차가...,0,낯선 사람,여성,남성
3,[CLS] name2 안녕 [SEP] 안녕 name1 [SEP] 날씨 좋다\n일하니...,1,낯선 사람,여성,남성
4,[CLS] [SEP] 요즘 날씨가 많이 선선해졌죠?!?! 아침엔 꽤나 쌀쌀하네요 오...,0,낯선 사람,여성,여성


In [35]:
malmungchi_4000_samples_label=malmungchi_4000_samples['label']
malmungchi_label_list=malmungchi_4000_samples_label.values.tolist()

In [36]:
malmungchi_4000_samples_data=malmungchi_4000_samples['data']
malmungchi_data_list=malmungchi_4000_samples_data.values.tolist()

In [37]:
for data in malmungchi_data_list:
    data=str(data)

In [38]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
tokenized_malmungchi_input = [tokenizer.tokenize(str(input)) for input in malmungchi_data_list]

In [39]:
MAX_LEN = 500

malmungchi_input_id = [tokenizer.convert_tokens_to_ids(token) for token in tokenized_malmungchi_input]
malmungchi_input_id = pad_sequences(malmungchi_input_id, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [40]:
malmungchi_attention_masks = []

for num in malmungchi_input_id:
    attention_sample_mask = [float(i>0) for i in num]
    malmungchi_attention_masks.append(attention_sample_mask)

In [41]:
malmungchi_x_train, malmungchi_x_test, malmungchi_y_train, malmungchi_y_test = train_test_split(malmungchi_input_id, malmungchi_label_list, random_state=2, test_size=0.2, shuffle=True)
malmungchi_mask_train, sample_mask_test, temp_train_id, temp_test_id= train_test_split(malmungchi_attention_masks, malmungchi_input_id, random_state=2, test_size=0.2, shuffle=True)     

In [42]:
malmungchi_x_val, malmungchi_x_eval, malmungchi_y_val, malmungchi_y_eval = train_test_split(malmungchi_x_test, malmungchi_y_test, test_size=0.1, random_state=2, shuffle=True)
malmungchi_mask_val, malmungchi_mask_eval, _, _ = train_test_split(sample_mask_test, temp_test_id, random_state=2, test_size=0.1, shuffle=True)   

In [43]:
malmungchi_train_inputs = torch.tensor(malmungchi_x_train)
malmungchi_train_labels = torch.tensor(malmungchi_y_train)
malmungchi_train_masks = torch.tensor(malmungchi_mask_train)

malmungchi_val_inputs = torch.tensor(malmungchi_x_val)
malmungchi_val_labels = torch.tensor(malmungchi_y_val)
malmungchi_val_masks = torch.tensor(malmungchi_mask_val)  

malmungchi_eval_inputs2 = torch.tensor(malmungchi_x_eval)
malmungchi_eval_labels2 = torch.tensor(malmungchi_y_eval)
malmungchi_eval_masks2 = torch.tensor(malmungchi_mask_eval)  

In [44]:
BATCH_SIZE = 16

malmungchi_train_data = TensorDataset(malmungchi_train_inputs, malmungchi_train_masks, malmungchi_train_labels)
malmungchi_train_sampler = RandomSampler(malmungchi_train_data)
malmungchi_train_dataloader = DataLoader(malmungchi_train_data, sampler=malmungchi_train_sampler, batch_size=BATCH_SIZE)

malmungchi_val_data = TensorDataset(malmungchi_val_inputs, malmungchi_val_masks, malmungchi_val_labels)
malmungchi_val_sampler = SequentialSampler(malmungchi_val_data)
malmungchi_val_dataloader = DataLoader(malmungchi_val_data, sampler=malmungchi_val_sampler, batch_size=BATCH_SIZE)

malmungchi_eval_data2 = TensorDataset(malmungchi_eval_inputs2, malmungchi_eval_masks2, malmungchi_eval_labels2)
malmungchi_eval_sampler2 = SequentialSampler(malmungchi_eval_data2)
malmungchi_eval_dataloader2 = DataLoader(malmungchi_eval_data2, sampler=malmungchi_eval_sampler2, batch_size=BATCH_SIZE)

## 5.2 Training

In [45]:
optimizer = AdamW(SemanticScorePrediction.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 30

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(malmungchi_train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [46]:
#랜덤시드 고정
seed_val = 14
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
SemanticScorePrediction.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    SemanticScorePrediction.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(malmungchi_train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(malmungchi_train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        for i in range(len(b_labels)):
            b_labels[i] += 1

        # Forward 수행                
        outputs = SemanticScorePrediction(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask,
                        turn_ids = None,
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(SemanticScorePrediction.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        SemanticScorePrediction.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(malmungchi_train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 30 ========
Training...

  Average training loss: 0.55
  Training epcoh took: 0:01:10

======== Epoch 2 / 30 ========
Training...

  Average training loss: 0.48
  Training epcoh took: 0:01:11

======== Epoch 3 / 30 ========
Training...

  Average training loss: 0.46
  Training epcoh took: 0:01:11

======== Epoch 4 / 30 ========
Training...

  Average training loss: 0.45
  Training epcoh took: 0:01:10

======== Epoch 5 / 30 ========
Training...

  Average training loss: 0.39
  Training epcoh took: 0:01:11

======== Epoch 6 / 30 ========
Training...

  Average training loss: 0.33
  Training epcoh took: 0:01:10

======== Epoch 7 / 30 ========
Training...

  Average training loss: 0.26
  Training epcoh took: 0:01:11

======== Epoch 8 / 30 ========
Training...

  Average training loss: 0.21
  Training epcoh took: 0:01:10

======== Epoch 9 / 30 ========
Training...

  Average training loss: 0.18
  Training epcoh took: 0:01:10

======== Epoch 10 / 30 ========
Training...



## 5.3 Validation

In [47]:
#Validation

logits_results = np.array([])
label_ids_results = np.array([])
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for batch in malmungchi_val_dataloader:
    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)

    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    for i in range(len(b_labels)):
        b_labels[i] += 1

    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = SemanticScorePrediction(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask,
                        turn_ids = None,
                        labels=b_labels)

    #로짓 구함
    logits = outputs[1]
    #print(logits)
    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    logits = np.argmax(logits, axis=1)

    logits_results = np.append(logits_results, logits)
    label_ids_results = np.append(label_ids_results, label_ids)
    

# 출력 로스과 라벨을 비교하여 정확도 계산
tmp_eval_accuracy = flat_accuracy(logits_results, label_ids_results)
eval_accuracy += tmp_eval_accuracy
nb_eval_steps += 1
acc, prec, recall, f1, RMSE = evaluation_metrics(logits_results, label_ids_results)

print('  [Accuracy]: {:.2f}%'.format(acc * 100))
print('  [Precision]: {:.2f}%'.format(prec * 100))
print('  [Recall]: {:.2f}%'.format(recall * 100))
print('  [F1_Score]: {:.2f}%'.format(f1 * 100))
print('  [RMSE]: {:.2f}'.format(RMSE))
print("  Validation took: {:}".format(format_time(time.time() - t0)))

  [Accuracy]: 82.79%
  [Precision]: 40.30%
  [Recall]: 45.94%
  [F1_Score]: 41.81%
  [RMSE]: 0.45
  Validation took: 0:01:15


## 5.4 Evaluation

In [50]:
#Evaluation
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
SemanticScorePrediction.eval()
score_list_final=[]

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
logits_results = np.array([])
label_ids_results = np.array([])

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(malmungchi_eval_dataloader2):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(malmungchi_eval_dataloader2), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    
    
    for i in range(len(b_labels)):
            b_labels[i] += 1
        
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = SemanticScorePrediction(b_input_ids, 
                        token_type_ids=None, 
                        turn_ids=None,
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]
    
    #score_list=relation_score_prediction(logits)
    #score_list_final.append(score_list)
    
    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    
    logits = np.argmax(logits, axis=1)

    logits_results = np.append(logits_results, logits)
    label_ids_results = np.append(label_ids_results, label_ids)
                                  
    #print("============================================================================================================Dialogue============================================================================================================")
    #print(malmungchi_data_list[:5])
    
    #print("=======================================================================================================Dialogue Integer Encoding=======================================================================================================")
    #print(b_input_ids[:5])
                                  
    #print("============================================================================================================argmax[logits]============================================================================================================")
    #print(logits[:5])
    
    #print("============================================================================================================Relation Score============================================================================================================")
    #print(score_list[:5])

# 출력 로스과 라벨을 비교하여 정확도 계산
tmp_eval_accuracy = flat_accuracy(logits_results, label_ids_results)
eval_accuracy += tmp_eval_accuracy
nb_eval_steps += 1
acc, prec, recall, f1, RMSE = evaluation_metrics(logits_results, label_ids_results)

print('  [Accuracy]: {:.2f}%'.format(acc * 100))
print('  [Precision]: {:.2f}%'.format(prec * 100))
print('  [Recall]: {:.2f}%'.format(recall * 100))
print('  [F1_Score]: {:.2f}%'.format(f1 * 100))
print('  [RMSE]: {:.2f}'.format(RMSE))
print("  Evaluation took: {:}".format(format_time(time.time() - t0)))


  [Accuracy]: 79.38%
  [Precision]: 57.56%
  [Recall]: 59.61%
  [F1_Score]: 58.26%
  [RMSE]: 0.45
  Evaluation took: 0:00:01
